# Randomly Sample Instance Set
Run this script once for each benchmark and commit the resulting instance set config files.
The fraction of subsampling should be similar to the number of instances selected by SELECTOR for lower thresholds.
Normally, samples 10% of the instance sets. 

In [3]:
from pathlib import Path
from rich import print as printr
from omegaconf import OmegaConf
from hydra.utils import instantiate
import numpy as np
from dacbench.abstract_benchmark import AbstractBenchmark

##################### DEFINE BENCHMARK AND INSTANCE SET
benchmark_cfg_fn = "../instance_dac/configs/benchmark/sigmoid.yaml"
instance_set_cfg_fn = "../instance_dac/configs/inst/sigmoid/2D3M_train.yaml"

# Subsample
seeds = [234235, 43534, 411, 799, 32544]
fraction = 0.1
#####################

# Instantiate benchmark and read instance set
bench_cfg = OmegaConf.load(benchmark_cfg_fn)
iset_cfg = OmegaConf.load(instance_set_cfg_fn)
bench_cfg = OmegaConf.merge(bench_cfg, iset_cfg)
benchmark = instantiate(bench_cfg.benchmark)

def random_sample_instance_set(benchmark: AbstractBenchmark, seed: int, fraction: float) -> dict:
    benchmark.read_instance_set()
    instance_set = benchmark.config["instance_set"]
    rng = np.random.default_rng(seed=seed)
    n_instances = len(instance_set)
    ids = rng.choice(list(instance_set.keys()), size=int(fraction * n_instances), replace=False)
    ids = [int(k) for k in ids]
    instance_set_new = {k: instance_set[k] for k in ids}
    return instance_set_new

for seed in seeds:
    instance_set_new = random_sample_instance_set(benchmark=benchmark, seed=seed, fraction=fraction)

    print("Size of new instance set:", len(instance_set_new))

    instance_set_id = iset_cfg.instance_set_id + f"_{fraction}_{seed}"

    cfg = OmegaConf.create({
        "instance_set_id": instance_set_id,
        "instance_set_selection": "random",
        "selector": {
            "seed": seed,
            "fraction": fraction,
        },
        "source_instance_set_id": iset_cfg.instance_set_id,
        "benchmark": {
            "config": {
                "instance_set": instance_set_new
            }
        },
        
    })
    yaml_str = OmegaConf.to_yaml(cfg)
    yaml_str = f"# @package _global_\n\n{yaml_str}"

    target_fn = Path("../instance_dac/configs/inst/") / bench_cfg.benchmark_id.lower() / "random" / f"{instance_set_id}.yaml"
    target_fn.parent.mkdir(exist_ok=True, parents=True)
    print("Wrote", target_fn)
    target_fn.write_text(yaml_str)

Size of new instance set: 30
Wrote ../instance_dac/configs/inst/sigmoid/random/2D3M_train_0.1_234235.yaml
Size of new instance set: 30
Wrote ../instance_dac/configs/inst/sigmoid/random/2D3M_train_0.1_43534.yaml
Size of new instance set: 30
Wrote ../instance_dac/configs/inst/sigmoid/random/2D3M_train_0.1_411.yaml
Size of new instance set: 30
Wrote ../instance_dac/configs/inst/sigmoid/random/2D3M_train_0.1_799.yaml
Size of new instance set: 30
Wrote ../instance_dac/configs/inst/sigmoid/random/2D3M_train_0.1_32544.yaml


In [4]:
# Check if it works:)
cfg = OmegaConf.load(target_fn)
bcfg =  OmegaConf.load(benchmark_cfg_fn)
cfg = OmegaConf.merge(bcfg, cfg)
benchmark = instantiate(cfg.benchmark)
env = benchmark.get_environment()
printr(len(env.instance_set))
printr(env.instance_set)

30

{299: [4.815429229461179, 2.7816176276037967, 4.439819850877722, 4.005233707279633], 287: [5.029538623940392, 
3.9300996226599487, 2.5009560669928828, 3.517955235435612], 32: [2.6935319003072093, 3.379288831385219, 
8.64088234978544, -1.4008456321176168], 194: [5.857836784234157, 1.0751211003457897, 2.212130908894809, 
-0.7317262284296113], 202: [0.3796041578885978, 2.381906558937664, 1.8255417461780605, 0.9174862767187082], 295: 
[2.814519854641466, 6.728191140435256, -1.7961905384813868, 3.2447303356112576], 237: [1.3603274108658878, 
4.291495475336782, 0.956284673197936, 5.441808301962239], 98: [4.797970213475937, 0.2494571060236206, 
-3.2402046096668595, 6.370806523830282], 169: [7.672872999001572, 4.990377192036222, 6.024183185414975, 
-1.9379220225563736], 119: [1.8212151723691101, 1.4780391432769244, 4.0685581802252315, 4.362072015355503], 270: 
[1.924026836405, 2.7580682605742703, 1.646214083209576, 1.996125161663582], 64: [5.177123079509854, 
1.7353960522471783, -0.27463776160156206, 0.5863207853372709], 130: [4.064244656792983, 4.7716813057767125, 
3.078758751578766, 1.9557658112159155], 89: [2.0780627610286864, 1.8004680027063937, 4.480202899003775, 
3.9514400400316814], 84: [0.4268809753789182, 2.2638888977935814, 1.4154376692990676, -1.3976092909669209], 2: 
[5.410536230957515, 5.700091608324946, -5.3540400976249165, 2.76787147719077], 43: [4.905227259654491, 
3.104873656179629, 3.560111104441587, 3.1936940390756687], 42: [3.4631061502388762, 3.264221200904144, 
1.0810847412013014, 1.4636998912875985], 161: [1.723824463909902, 3.255052055298451, 8.673977983130346, 
0.7776783896857395], 90: [3.441776026766, 4.310033415306151, -4.3548487637334885, -0.6080528997473085], 282: 
[0.5822748992498026, 5.512159870662867, -1.168948223279786, 7.149733425876296], 77: [4.607976014008139, 
-0.23720035078244006, 3.209175931615208, 7.41920537379076], 216: [5.281477624730198, 4.042676458366114, 
5.714603736550742, -1.8536927899278695], 279: [3.5973577950453532, 5.211014816491561, -0.44196741114150484, 
0.5687055142869353], 80: [6.827823178794205, 8.727401615002458, 1.5530047463846481, -1.9015760884779742], 93: 
[6.085867629073292, 6.192285512290833, 1.3613578420758399, 11.03131168513633], 203: [2.7693020222718907, 
3.2681075670629283, 1.8703226414448288, 3.9794167745837434], 67: [3.190730385781652, 6.659409454273963, 
1.1217798512803276, 2.7437717740148555], 87: [2.9425049950565123, 5.7606383061778015, 1.6790460570929602, 
0.28667876559338556], 149: [5.406767910709146, 4.854325379155132, -6.404191877907916, 0.3611679915156436]}